In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('euw.csv')

In [3]:
df.drop(df.columns[0], axis=1, inplace=True)

In [4]:
x_train = df[df.columns[1:]]
y_train = df["result"]

In [5]:
x_train.shape

(2865, 23)

In [6]:
y_train = to_categorical(y_train, 2)

In [7]:
y_train[1002]

array([1., 0.], dtype=float32)